In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from nba_api.stats.endpoints import scoreboardv2
from ___team_data import players_by_team

In [2]:
# ============================================
# Define safe functions
def inv(x):
    return 1.0 / x if x != 0 else 0

# === Your best PySR frozen expression ===
def predict_points(row):
    try:
        adjusted_shot_creation = row['adjusted_shot_creation']
        delta_fga_vs_rolling = row['delta_fga_vs_rolling']
        fg_pct_vs_avg_delta = row['fg_pct_vs_avg_delta']
        rolling_pts_from_ft_5g = row['rolling_pts_from_ft_5g']
        rolling_usg_pct_5G = row['rolling_usg_pct_5G']
        volume_efficiency_to_pts = row['volume_efficiency_to_pts']

        result = np.ceil(
            (
                (
                    fg_pct_vs_avg_delta
                    * (
                        adjusted_shot_creation
                        + max(
                            (rolling_usg_pct_5G ** -0.6628592) / inv(volume_efficiency_to_pts),
                            round(
                                inv(
                                    np.exp(
                                        -2.7052715
                                        + min(
                                            delta_fga_vs_rolling - -0.8848275,
                                            (0.76128376 ** (rolling_pts_from_ft_5g - -0.15936984))
                                            / inv(np.sqrt(adjusted_shot_creation))
                                        )
                                    )
                                )
                            )
                        )
                        * delta_fga_vs_rolling
                    )
                ) ** 0.8927275
                + -0.08081033
            )
            * 0.9920129
        )
        return result
    except Exception as e:
        print(f"[!] Prediction error: {e}")
        return np.nan



def floor_to_nearest_5(x):
    return np.floor(x / 5) * 5

def get_today_team_ids():
    today = datetime.today().strftime('%Y-%m-%d')
    games = scoreboardv2.ScoreboardV2(game_date=today).get_normalized_dict()
    team_ids = set()
    for game in games['GameHeader']:
        team_ids.add(game['HOME_TEAM_ID'])
        team_ids.add(game['VISITOR_TEAM_ID'])
    return team_ids

# ============================================

In [3]:
if __name__ == "__main__":
    # Load today's teams
    today_team_ids = get_today_team_ids()
    print(f"Teams playing today: {today_team_ids}")

    # Load full feature set
    features_df = pd.read_csv("predictions_with_pysr.csv")
    features_df = features_df.drop_duplicates(subset=["PLAYER_NAME", "GAME_DATE"]).sort_values(by=["PLAYER_NAME", "GAME_DATE"])

    # Flatten list of all players
    target_players = [player for team_players in players_by_team.values() for player in team_players]

    # Filter features to only target players
    candidate_df = features_df[features_df['PLAYER_NAME'].isin(target_players)].copy()

    # Keep only players whose last TEAM_ID is playing today
    latest_game_df = candidate_df.sort_values('GAME_DATE').drop_duplicates('PLAYER_NAME', keep='last')

    playing_today_df = latest_game_df[latest_game_df['TEAM_ID'].isin(today_team_ids)].copy()

    if playing_today_df.empty:
        print("[!] No players from your list have games today.")
    else:
        print(f"[✓] {len(playing_today_df)} players from your list have games today.")

        # Apply prediction
        playing_today_df['predicted_points'] = playing_today_df.apply(predict_points, axis=1)
        playing_today_df['floored_prediction'] = playing_today_df['predicted_points'].apply(floor_to_nearest_5)

        # Final output
        output_cols = ['PLAYER_NAME', 'GAME_DATE', 'predicted_points', 'floored_prediction']
        print(playing_today_df[output_cols].sort_values(by=['PLAYER_NAME']))

        # Save to file
        playing_today_df[output_cols].to_csv("today_predictions.csv", index=False)
        print("[✓] Today's predictions saved to 'today_predictions.csv'")

Teams playing today: {1610612743, 1610612746, 1610612747, 1610612749, 1610612750, 1610612752, 1610612754, 1610612765}
[✓] 39 players from your list have games today.
                   PLAYER_NAME   GAME_DATE  predicted_points  \
3190              Aaron Gordon  2025-04-13              22.0   
4682               Amir Coffey  2025-04-13              -0.0   
3394           Anthony Edwards  2025-04-13              40.0   
1352            Ausar Thompson  2025-04-11              14.0   
4860             Austin Reaves  2025-04-11              31.0   
1298           Cade Cunningham  2025-04-11              43.0   
3144           Christian Braun  2025-04-13              15.0   
4546         Derrick Jones Jr.  2025-04-13               5.0   
3725          Donte DiVincenzo  2025-04-13              18.0   
5023       Dorian Finney-Smith  2025-04-11              14.0   
1637            Isaiah Stewart  2025-04-10              -0.0   
4419               Ivica Zubac  2025-04-13              24.0   
35